# News Topic Modeling 
Cluster News

https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

### Import framworks

In [33]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

from pprint import  pprint
import numpy as np
import pandas as pd
import re

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import spacy

import pyLDAvis
import pyLDAvis.gensim


### Prpare stopwords

In [2]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

### Read data

In [3]:
df = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')
df.head()

,content,target,target_names
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,rec.autos
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4,comp.sys.mac.hardware
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1,comp.graphics
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14,sci.space


In [4]:
print(df.target_names.unique())

['rec.autos' 'comp.sys.mac.hardware' 'comp.graphics' 'sci.space'
 'talk.politics.guns' 'sci.med' 'comp.sys.ibm.pc.hardware'
 'comp.os.ms-windows.misc' 'rec.motorcycles' 'talk.religion.misc'
 'misc.forsale' 'alt.atheism' 'sci.electronics' 'comp.windows.x'
 'rec.sport.hockey' 'rec.sport.baseball' 'soc.religion.christian'
 'talk.politics.mideast' 'talk.politics.misc' 'sci.crypt']


### Cleaning : Remove emails and new line characters 

In [5]:
data = df.content.values.tolist()
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data] #email
data = [re.sub('\s+', ' ', sent) for sent in data] # new line character
data = [re.sub("\'", '', sent) for sent in data]
pprint(data[:1])

['From: (wheres my thing) Subject: WHAT car is this!? Nntp-Posting-Host: '
 'rac3.wam.umd.edu Organization: University of Maryland, College Park Lines: '
 '15 I was wondering if anyone out there could enlighten me on this car I saw '
 'the other day. It was a 2-door sports car, looked to be from the late 60s/ '
 'early 70s. It was called a Bricklin. The doors were really small. In '
 'addition, the front bumper was separate from the rest of the body. This is '
 'all I know. If anyone can tellme a model name, engine specs, years of '
 'production, where this car is made, history, or whatever info you have on '
 'this funky looking car, please e-mail. Thanks, - IL ---- brought to you by '
 'your neighborhood Lerxst ---- ']


### Tokenize Words and Clean-up text
Tokanize each sentense in to list of words, removing punctuation and unnecessory characters altogether. <br>
Gensim's simple_preprocess() is the great option. set deacc=True to remove punctuations. 

In [6]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(sentence, deacc=True))
        
data_words = list(sent_to_words(data))
print(data_words[:1])

[['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp', 'posting', 'host', 'rac', 'wam', 'umd', 'edu', 'organization', 'university', 'of', 'maryland', 'college', 'park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']]


## Creating Bigram and Trigram models

In [7]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # More threshold fewer phrases
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)

In [8]:
trigram[bigram[data_words[0]]]

['from',
 'wheres',
 'my',
 'thing',
 'subject',
 'what',
 'car',
 'is',
 'this',
 'nntp_posting_host',
 'rac_wam_umd_edu',
 'organization',
 'university',
 'of',
 'maryland_college_park',
 'lines',
 'was',
 'wondering',
 'if',
 'anyone',
 'out',
 'there',
 'could',
 'enlighten',
 'me',
 'on',
 'this',
 'car',
 'saw',
 'the',
 'other',
 'day',
 'it',
 'was',
 'door',
 'sports',
 'car',
 'looked',
 'to',
 'be',
 'from',
 'the',
 'late',
 'early',
 'it',
 'was',
 'called',
 'bricklin',
 'the',
 'doors',
 'were',
 'really',
 'small',
 'in',
 'addition',
 'the',
 'front_bumper',
 'was',
 'separate',
 'from',
 'the',
 'rest',
 'of',
 'the',
 'body',
 'this',
 'is',
 'all',
 'know',
 'if',
 'anyone',
 'can',
 'tellme',
 'model',
 'name',
 'engine',
 'specs',
 'years',
 'of',
 'production',
 'where',
 'this',
 'car',
 'is',
 'made',
 'history',
 'or',
 'whatever',
 'info',
 'you',
 'have',
 'on',
 'this',
 'funky',
 'looking',
 'car',
 'please',
 'mail',
 'thanks',
 'il',
 'brought',
 'to',
 

In [9]:
bigram[data_words[0]]

['from',
 'wheres',
 'my',
 'thing',
 'subject',
 'what',
 'car',
 'is',
 'this',
 'nntp_posting',
 'host',
 'rac_wam',
 'umd_edu',
 'organization',
 'university',
 'of',
 'maryland_college',
 'park',
 'lines',
 'was',
 'wondering',
 'if',
 'anyone',
 'out',
 'there',
 'could',
 'enlighten',
 'me',
 'on',
 'this',
 'car',
 'saw',
 'the',
 'other',
 'day',
 'it',
 'was',
 'door',
 'sports',
 'car',
 'looked',
 'to',
 'be',
 'from',
 'the',
 'late',
 'early',
 'it',
 'was',
 'called',
 'bricklin',
 'the',
 'doors',
 'were',
 'really',
 'small',
 'in',
 'addition',
 'the',
 'front_bumper',
 'was',
 'separate',
 'from',
 'the',
 'rest',
 'of',
 'the',
 'body',
 'this',
 'is',
 'all',
 'know',
 'if',
 'anyone',
 'can',
 'tellme',
 'model',
 'name',
 'engine',
 'specs',
 'years',
 'of',
 'production',
 'where',
 'this',
 'car',
 'is',
 'made',
 'history',
 'or',
 'whatever',
 'info',
 'you',
 'have',
 'on',
 'this',
 'funky',
 'looking',
 'car',
 'please',
 'mail',
 'thanks',
 'il',
 'brough

In [10]:
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [11]:
# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp_posting_host', 'rac_wam_umd_edu', 'organization', 'university', 'of', 'maryland_college_park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front_bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']


In [12]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [13]:
data_words_nostops = remove_stopwords(data_words)

data_words_bigrams = make_bigrams(data_words_nostops)

nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

data_lemmatized = lemmatization(data_words_bigrams)


In [14]:
print(data_lemmatized[:1])

[['where', 'thing', 'car', 'nntp_poste', 'host', 'park', 'line', 'wonder', 'could', 'enlighten', 'car', 'see', 'day', 'door', 'sport', 'car', 'look', 'late', 'early', 'call', 'bricklin', 'door', 'really', 'small', 'addition', 'separate', 'rest', 'body', 'know', 'tellme', 'model', 'name', 'engine', 'year', 'production', 'car', 'make', 'history', 'info', 'funky', 'look', 'car', 'mail', 'thank', 'bring', 'neighborhood', 'lerxst']]


In [15]:
id2word = corpora.Dictionary(data_lemmatized)

corpus = [id2word.doc2bow(text) for text in data_lemmatized]

corpus[:1]

[[(0, 1),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 5),
  (6, 1),
  (7, 1),
  (8, 2),
  (9, 1),
  (10, 1),
  (11, 1),
  (12, 1),
  (13, 1),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 1),
  (20, 2),
  (21, 1),
  (22, 1),
  (23, 1),
  (24, 1),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 1),
  (29, 1),
  (30, 1),
  (31, 1),
  (32, 1),
  (33, 1),
  (34, 1),
  (35, 1),
  (36, 1),
  (37, 1),
  (38, 1),
  (39, 1),
  (40, 1)]]

In [16]:
id2word[0]

'addition'

## Model
Apart from that, alpha and eta are hyperparameters that affect sparsity of the topics. According to the Gensim docs, both defaults to 1.0/num_topics prior.

chunksize is the number of documents to be used in each training chunk. update_every determines how often the model parameters should be updated and passes is the total number of training passes.

In [24]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                            num_topics=20,
                                           random_state=0,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [25]:
lda_model.print_topics()


[(0,
  '0.170*"encryption" + 0.080*"animal" + 0.049*"privacy" + 0.027*"wiretap" + 0.026*"label" + 0.022*"fish" + 0.008*"inherent" + 0.005*"ms_window" + 0.004*"negligent" + 0.003*"efficiently"'),
 (1,
  '0.166*"player" + 0.108*"season" + 0.097*"fan" + 0.077*"trade" + 0.032*"lose" + 0.028*"penguin" + 0.027*"implementation" + 0.022*"leafs" + 0.016*"captain" + 0.015*"flyer"'),
 (2,
  '0.031*"report" + 0.026*"physical" + 0.024*"result" + 0.023*"system" + 0.021*"purpose" + 0.018*"effect" + 0.018*"increase" + 0.018*"study" + 0.017*"produce" + 0.017*"accept"'),
 (3,
  '0.188*"ax" + 0.159*"max" + 0.053*"village" + 0.038*"greek" + 0.037*"turkish" + 0.028*"armenian" + 0.018*"inhabitant" + 0.016*"closed" + 0.014*"genocide" + 0.012*"electrical"'),
 (4,
  '0.072*"file" + 0.059*"window" + 0.056*"program" + 0.034*"image" + 0.030*"entry" + 0.030*"version" + 0.029*"color" + 0.027*"application" + 0.023*"display" + 0.022*"set"'),
 (5,
  '0.117*"motif" + 0.048*"own" + 0.033*"tip" + 0.027*"giant" + 0.015*"s

In [27]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='u_mass')



Perplexity:  -12.905476419887073


In [28]:
print('\nCoherence Score: ', coherence_model_lda.get_coherence())


Coherence Score:  -4.28608617942503


In [34]:
pyLDAvis.enable_notebook()

vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)

//anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [35]:
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
11    -0.327337 -0.051616       1        1  36.930771
7     -0.282325 -0.203029       2        1  12.524016
10    -0.251515 -0.007995       3        1  10.725438
19    -0.216397  0.020682       4        1   8.456800
13    -0.211279  0.269317       5        1   8.240979
12    -0.119446 -0.222153       6        1   4.817239
2     -0.073089  0.085978       7        1   3.825294
4     -0.068936  0.136015       8        1   3.164776
6      0.033793  0.169922       9        1   1.687477
17     0.015200 -0.007249      10        1   1.591330
14     0.044995 -0.050633      11        1   1.323529
3      0.149054 -0.029216      12        1   1.320315
8      0.093465 -0.022836      13        1   1.304285
16     0.123683 -0.024982      14        1   1.154424
9      0.117095 -0.027490      15        1   1.131852
18     0.109688  0.083335      16        1   1.037660
1      0.193764 -0.031326      17        1   0.365025
0      0.217467 -0.029429      18        1   0.215615
15     0.226183 -0.028637      19        1   0.094804
5      0.225936 -0.028658      20        1   0.088374, topic_info=     Category          Freq          Term         Total  loglift  logprob
19    Default  15754.000000          line  15754.000000  30.0000  30.0000
174   Default  16073.000000         would  16073.000000  29.0000  29.0000
40    Default   4605.000000          year   4605.000000  28.0000  28.0000
197   Default  13331.000000         write  13331.000000  27.0000  27.0000
3685  Default   2799.000000            ax   2799.000000  26.0000  26.0000
...       ...           ...           ...           ...      ...      ...
3323  Topic20      0.019234      override      1.186446   2.9093 -10.8567
6727  Topic20      0.019234    distribute      1.186473   2.9093 -10.8567
4205  Topic20      0.019234     implement      1.186491   2.9093 -10.8567
5605  Topic20      0.019234  announcement      1.186500   2.9093 -10.8567
3395  Topic20      0.019234        recent      1.186498   2.9093 -10.8567

[891 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
472      11  0.993974      absolute
41       16  0.971504  acceleration
1211      7  0.749334        accept
1211     11  0.249778        accept
1288      7  0.994777       acquire
...     ...       ...           ...
197       1  0.999903         write
40        1  0.527579          year
40        3  0.240341          year
40        4  0.001086          year
40       10  0.231006          year

[1102 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[12, 8, 11, 20, 14, 13, 3, 5, 7, 18, 15, 4, 9, 17, 10, 19, 2, 1, 16, 6])

## Building LDA Mallet Model